In [17]:
from  ase.io.espresso import  read_espresso_out
import numpy as np
import inspect

In [18]:
## 4つのQE　outputからBECを計算するコード．
def extract_BEC(directory,prefix, efield):
    ##=================================
    #     input 
    # ~~~~~~~~~~~~~
    # directory: str
    #     QE output file directory(path)
    # prefix: str
    #     QE output file name. prefix.pw.out/ prefix_x.pw.out/ etc...
    #
    # efield: double
    #      applied electric field in Ry a.u. (same as QE)
    #
    #     output
    # ~~~~~~~~~~~~~
    # BEC: 3D-array
    #      BEC(atom_id, BEC_x, BEC_y)
    ##=================================
    #
    from  ase.io.espresso import  read_espresso_out
    import numpy as np
    #
    ## unit conversion 
    # In ASE, unit is converted from QEoutput Ry/bohr to eV/Ang.
    # We have to back to QE units.
    # Ry: 13.605691932782346 eV
    # Bohr: 0.5291772083535413 Ang
    # above values are got from 
    # ==========================
    # from ase.units import create_units
    # units = create_units('2006')
    # ==========================
    conv=  0.5291772083535413/13.605691932782346 
    
    # output filename
    filename0=directory+"/"+prefix+".pw.out"
    filenamex=directory+"/"+prefix+"_x.pw.out"
    filenamey=directory+"/"+prefix+"_y.pw.out"
    filenamez=directory+"/"+prefix+"_z.pw.out"
    #
    ##=================================
    # extract data from QEoutput
    ##=================================
    #
    ## reference
    # read output into ASE Atoms objects
    with open(filename0, "r") as f:
        traj0 = list(read_espresso_out(f, index=slice(None),results_required=True))
    # extract force    
    for step in traj0[::-1]:
        test0=step.get_forces()*conv
        # num of atoms
        NUM_ATOM=step.get_global_number_of_atoms()
        print("NUM_ATOM::", NUM_ATOM)
    ###################################
    ## x-axis
    with open(filenamex, "r") as f:
        trajx = list(read_espresso_out(f, index=slice(None),results_required=True))
    for step in trajx[::-1]:
        testx=step.get_forces()*conv
    ###################################
    ## y-axis
    with open(filenamey, "r") as f:
        trajy = list(read_espresso_out(f, index=slice(None),results_required=True))
    for step in trajy[::-1]:
        testy=step.get_forces()*conv
    ###################################
    ## z-axis
    with open(filenamez, "r") as f:
        trajz = list(read_espresso_out(f, index=slice(None),results_required=True))
    for step in trajz[::-1]:
        testz=step.get_forces()*conv
    #
    ##=================================
    # calculate BEC 
    ##=================================
    # BEC=delta_force/dE
    # electric field E=0.001 Ry a.u. 
    force_del_x=(testx-test0)/efield
    force_del_y=(testy-test0)/efield
    force_del_z=(testz-test0)/efield
    # 3D-array for BEC.
    BEC=np.zeros((NUM_ATOM,3,3))
    # loop of atom ID
    for i in range(0,NUM_ATOM):
        BEC[i,0,:]= force_del_x[i,:]
        BEC[i,1,:]=force_del_y[i,:]
        BEC[i,2,:]=force_del_z[i,:]    
    #    
    # ASR:: calculate sum of BEC
    #####
    # (これがちょっと怪しいわ．．． 既に最小の桁になっちゃって誤差が丸まってるよねこれ．）
    #####
    BEC_SUM=np.sum(BEC, axis=0)
    #
    # ASR:: subtract BEC_SUM from BEC
    BEC=BEC-BEC_SUM/NUM_ATOM
    #
    # convert unit from Ry atomic units to Hretree atomic units.
    # e[Ry]=sqrt(2)[Hartree]
    BEC=BEC/np.sqrt(2)
    #print(BEC)
    return BEC
    

In [12]:
# 1: 8ben_mergeからoutputファイルを読み込む．
# 2: 1構造1ファイルとして，8ben_BECに保存．
#
#
for count in range(0,4000):
    # zero padding
    count_zero = str(count).zfill(4)
    print(count_zero)
    output=extract_BEC("8ben_merge/"+count_zero,"BEN_"+count_zero, 0.001)
    np.save("8ben_BEC/"+count_zero, output)
    

1969
NUM_ATOM:: 96


In [32]:
output=extract_BEC("../1WAT/water_bec_force","h2o_bec",0.001)

NUM_ATOM:: 3


In [33]:
print(output)

[[[-5.91650386e-01  6.47332688e-02 -4.36049182e-04]
  [ 6.14405082e-02 -5.65687782e-01  3.74766594e-04]
  [ 2.05532371e-03  2.57858273e-03 -7.23250029e-01]]

 [[ 2.78769777e-01 -4.87927249e-02  1.86204786e-04]
  [-4.90095710e-02  3.05559696e-01 -1.83847763e-04]
  [ 2.59272486e-04 -2.30045406e-03  3.61331565e-01]]

 [[ 3.12880609e-01 -1.59405439e-02  2.49844396e-04]
  [-1.24309372e-02  2.60128086e-01 -1.90918831e-04]
  [-2.31459620e-03 -2.78128667e-04  3.61918464e-01]]]


In [30]:
## Field=0のoutputからpositionsを取得するコード．
## もちろんinputからでも問題ないのだが．．．
def extract_positions(directory,prefix):
    ##=================================
    #     input 
    # ~~~~~~~~~~~~~
    # directory: str
    #     QE output file directory(path)
    # prefix: str
    #     QE output file name. prefix.pw.out/ prefix_x.pw.out/ etc...
    #
    #     output
    # ~~~~~~~~~~~~~
    # BEC: 3D-array
    #      BEC(atom_id, BEC_x, BEC_y)
    ##=================================
    #
    from  ase.io.espresso import  read_espresso_out
    import numpy as np
    #
    ## unit conversion 
    # In ASE, unit is converted from QEoutput Ry/bohr to eV/Ang.
    # We have to back to QE units.
    # Ry: 13.605691932782346 eV
    # Bohr: 0.5291772083535413 Ang
    # above values are got from 
    # ==========================
    # from ase.units import create_units
    # units = create_units('2006')
    # ==========================
    conv=  0.5291772083535413/13.605691932782346 
    
    # output filename
    filename0=directory+"/"+prefix+".pw.out"
    #
    ##=================================
    # extract data from QEoutput
    ##=================================
    #
    ## reference
    # read output into ASE Atoms objects
    with open(filename0, "r") as f:
        traj0 = list(read_espresso_out(f, index=slice(None),results_required=True))
    # extract positions
    for step in traj0[::-1]:
        test0=step.get_positions()
        # num of atoms
        NUM_ATOM=step.get_global_number_of_atoms()
        print("NUM_ATOM::", NUM_ATOM)
    return test0
    

In [36]:
## 取得したpositionsたちから差分を計算するコード．
def diff_positions():
    ##=================================
    #     input 
    # ~~~~~~~~~~~~~
    # directory: str
    #     QE output file directory(path)
    # prefix: str
    #     QE output file name. prefix.pw.out/ prefix_x.pw.out/ etc...
    #
    #     output
    # ~~~~~~~~~~~~~
    # BEC: 3D-array
    #      BEC(atom_id, BEC_x, BEC_y)
    ##=================================
    #
    from  ase.io.espresso import  read_espresso_out
    import numpy as np
    #
    ## unit conversion 
    # In ASE, unit is converted from QEoutput Ry/bohr to eV/Ang.
    # We have to back to QE units.
    # Ry: 13.605691932782346 eV
    # Bohr: 0.5291772083535413 Ang
    # above values are got from 
    # ==========================
    # from ase.units import create_units
    # units = create_units('2006')
    # ==========================
    
    conv=  0.5291772083535413/13.605691932782346 
    
    
    # ４０００構造の配列を作成する．
    NUM_STRUC=11
    # 4000*(3*96)の配列．
    configuration=np.zeros((NUM_STRUC,24,3))
    for count in range(0,NUM_STRUC):
        ## i番目のconfiguration
        # zero padding
        count_zero = str(count).zfill(2)
        print(count_zero)
        
        # directory setting
        directory="2ben_out2/"+count_zero
        prefix="ben2_"+count_zero
        
        # extract positions
        configuration[count,:,:]=extract_positions(directory,prefix)
        

    # 配列の差分をとる．
    # 
    diff_config=np.diff(configuration, axis=0, n = 1)
    # for debug
    print("diff_config.shape:: ", diff_config.shape)
    
    # 差分がcell_parameterの半分より大きければ修正する．
    # +-両方への修正が必要．
    print(diff_config[diff_config > 5])
    print(diff_config[diff_config < -5])

    # 上のprintで大丈夫そうなので，np.whereで置換を実行．
    #diff_config2=np.where(diff_config >5 , diff_config-10.58890000000000, diff_config)
    #diff_config3=np.where(diff_config2 <-5 , diff_config2+10.58890000000000, diff_config2)

    return diff_config
    

In [37]:
# ファイルに保存する．
tmp=diff_positions()
np.save("2ben_STR/diff_structure.npy",tmp)

00
NUM_ATOM:: 24
01
NUM_ATOM:: 24
02
NUM_ATOM:: 24
03
NUM_ATOM:: 24
04
NUM_ATOM:: 24
05
NUM_ATOM:: 24
06
NUM_ATOM:: 24
07
NUM_ATOM:: 24
08
NUM_ATOM:: 24
09
NUM_ATOM:: 24
10
NUM_ATOM:: 24
diff_config.shape::  (10, 24, 3)
[]
[]


In [38]:
print(tmp)

[[[0.        0.        0.       ]
  [0.        0.        0.       ]
  [0.        0.        0.       ]
  [0.        0.        0.       ]
  [0.        0.        0.       ]
  [0.        0.        0.       ]
  [0.        0.        0.       ]
  [0.        0.        0.       ]
  [0.        0.        0.       ]
  [0.        0.        0.       ]
  [0.        0.        0.       ]
  [0.        0.        0.       ]
  [0.        0.        0.1000005]
  [0.        0.        0.1000005]
  [0.        0.        0.1000005]
  [0.        0.        0.1000005]
  [0.        0.        0.1000005]
  [0.        0.        0.1000005]
  [0.        0.        0.1000005]
  [0.        0.        0.1000005]
  [0.        0.        0.1000005]
  [0.        0.        0.1000005]
  [0.        0.        0.1000005]
  [0.        0.        0.1000005]]

 [[0.        0.        0.       ]
  [0.        0.        0.       ]
  [0.        0.        0.       ]
  [0.        0.        0.       ]
  [0.        0.        0.       ]
  [0.       

In [45]:
idx = np.unravel_index(np.argmin(tmp), tmp.shape)
print(idx)
# (1, 0)

(399, 57, 0)


In [46]:
tmp[399, 57, 0]

-10.629999725938566

In [34]:
count_zero="0000"
test0=extract_positions("8ben_merge/"+count_zero,"BEN_"+count_zero)
# cartesian coordinates of atoms in Angstrom ?

NUM_ATOM:: 96


In [35]:
count_zero="0001"
test1=extract_positions("8ben_merge/"+count_zero,"BEN_"+count_zero)
# cartesian coordinates of atoms in Angstrom ?

NUM_ATOM:: 96


In [37]:
print(test1-test0)

[[ 0.          0.0099991  -0.01000016]
 [ 0.          0.01000016  0.01000016]
 [ 0.01000016 -0.01000016  0.        ]
 [ 0.01000016 -0.01000016 -0.0099991 ]
 [ 0.01000016  0.          0.        ]
 [ 0.         -0.01000016  0.        ]
 [-0.02000031  0.02000031 -0.01000016]
 [ 0.01000016 -0.01000016  0.02000031]
 [ 0.01000016 -0.01000016 -0.01000016]
 [-0.01000016 -0.03000047 -0.01000016]
 [-0.01000016 -0.01000016  0.01999926]
 [-0.01000016  0.          0.        ]
 [-0.01000016  0.01000016 -0.01000016]
 [ 0.          0.          0.        ]
 [ 0.01000016  0.         -0.01000016]
 [ 0.          0.          0.01000016]
 [-0.01000016  0.          0.        ]
 [ 0.          0.01000016  0.01000016]
 [ 0.         -0.01999926 -0.01000016]
 [ 0.02000031 -0.02000031  0.        ]
 [ 0.         -0.01000016 -0.0099991 ]
 [-0.01000016 -0.01999926  0.01000016]
 [-0.0099991  -0.01000016 -0.0099991 ]
 [-0.01000016  0.01000016  0.02000031]
 [-0.01000016  0.          0.01000016]
 [ 0.01000016  0.        

In [39]:
print(tmp[0,:,:])

[[ 0.          0.0099991  -0.01000016]
 [ 0.          0.01000016  0.01000016]
 [ 0.01000016 -0.01000016  0.        ]
 [ 0.01000016 -0.01000016 -0.0099991 ]
 [ 0.01000016  0.          0.        ]
 [ 0.         -0.01000016  0.        ]
 [-0.02000031  0.02000031 -0.01000016]
 [ 0.01000016 -0.01000016  0.02000031]
 [ 0.01000016 -0.01000016 -0.01000016]
 [-0.01000016 -0.03000047 -0.01000016]
 [-0.01000016 -0.01000016  0.01999926]
 [-0.01000016  0.          0.        ]
 [-0.01000016  0.01000016 -0.01000016]
 [ 0.          0.          0.        ]
 [ 0.01000016  0.         -0.01000016]
 [ 0.          0.          0.01000016]
 [-0.01000016  0.          0.        ]
 [ 0.          0.01000016  0.01000016]
 [ 0.         -0.01999926 -0.01000016]
 [ 0.02000031 -0.02000031  0.        ]
 [ 0.         -0.01000016 -0.0099991 ]
 [-0.01000016 -0.01999926  0.01000016]
 [-0.0099991  -0.01000016 -0.0099991 ]
 [-0.01000016  0.01000016  0.02000031]
 [-0.01000016  0.          0.01000016]
 [ 0.01000016  0.        

In [40]:
print(tmp[0,:,:]-(test1-test0))

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0.

In [59]:
output=extract_BEC("8ben_merge/0016","BEN_0016", 0.001)

NUM_ATOM:: 96


In [61]:
print(output)

[[[-7.34821684e-02  4.99909763e-03 -1.08119573e-02]
  [ 9.85751047e-03 -6.56479409e-02  1.02621892e-01]
  [ 1.22121761e-02  1.41788757e-01 -1.11708656e-01]]

 [[ 2.27126381e-02 -4.07648532e-02  1.11107394e-01]
  [-6.55624988e-02 -1.66085388e-01  5.11940154e-02]
  [ 1.05903825e-01  6.32786912e-02 -6.50891056e-02]]

 [[ 1.20952088e-03 -6.06699091e-02 -6.76730652e-03]
  [-8.55736207e-02 -5.95385383e-02  1.29880858e-01]
  [-2.89869586e-03  9.76935783e-02 -1.27724624e-01]]

 [[-1.20469414e-01 -2.62762353e-02  7.64167352e-02]
  [-5.31315616e-02 -1.33508979e-01  9.19445792e-02]
  [ 3.38496436e-02  1.28728495e-01  8.16638358e-02]]

 [[ 4.06095107e-02 -4.21578536e-02  7.67420043e-02]
  [ 4.79069264e-02 -1.19826462e-01  4.37693942e-02]
  [ 4.25117016e-02  2.63535751e-02 -1.14141103e-01]]

 [[-1.24729953e-02 -5.29978006e-02 -1.21201049e-02]
  [-9.98006092e-02 -5.64979791e-02  8.30208916e-02]
  [ 2.70967738e-02  3.19609319e-02 -1.07529655e-01]]

 [[ 8.93645233e-02 -3.97890459e-02 -1.76638220e-02]


In [8]:
# BECのtxtデータの整形と保存
test=np.loadtxt("8ben_DFPT_out/0200/BEN2_0200.txt")
#print(test)


# reshape
NUM_ATOM=96
BEC=test.reshape((NUM_ATOM,3,3))
print(BEC)

[[[-6.5200e-02 -4.2270e-02 -1.3795e-01]
  [-1.7320e-02 -1.6190e-01  1.2825e-01]
  [-2.8140e-02  9.6320e-02 -3.9030e-02]]

 [[-2.4550e-02 -5.3230e-02  7.0540e-02]
  [-7.1670e-02 -1.6346e-01  9.9000e-03]
  [ 1.0698e-01  2.2500e-02 -8.3000e-03]]

 [[-1.1540e-01 -1.7308e-01  6.5350e-02]
  [-1.1785e-01 -2.5610e-02  1.3380e-02]
  [ 5.3470e-02  1.7870e-02 -5.2460e-02]]

 [[-7.5220e-02 -4.7470e-02 -1.2200e-03]
  [-7.2700e-02 -1.5213e-01  6.6550e-02]
  [-5.8840e-02  1.4332e-01 -2.6780e-02]]

 [[-7.1980e-02 -5.7360e-02  5.8300e-02]
  [ 3.4650e-02 -1.5866e-01  1.2352e-01]
  [ 7.0860e-02  1.0820e-02 -1.2770e-02]]

 [[ 4.1330e-02 -7.6070e-02  2.2070e-02]
  [-1.7286e-01 -6.4600e-02  1.0080e-02]
  [ 1.4370e-02  3.9690e-02 -2.9000e-02]]

 [[ 8.1550e-02  3.1260e-02  9.6250e-02]
  [ 4.5780e-02  1.1016e-01 -1.3989e-01]
  [ 7.1920e-02 -1.1935e-01 -4.4720e-02]]

 [[ 5.6190e-02  2.2380e-02 -1.2505e-01]
  [ 3.7580e-02  1.5087e-01 -5.5800e-03]
  [-1.3909e-01 -1.4310e-02 -5.5900e-03]]

 [[ 6.0940e-02  1.6472e-

In [45]:
## 4つのQE　outputからBECを計算するコード．
def extract_BEC_ph(directory,prefix):
    ##=================================
    #     input 
    # ~~~~~~~~~~~~~
    # directory: str
    #     QE output file directory(path)
    # prefix: str
    #     QE output file name. prefix.pw.out/ prefix_x.pw.out/ etc...
    #
    # efield: double
    #      applied electric field in Ry a.u. (same as QE)
    #
    #     output
    # ~~~~~~~~~~~~~
    # BEC: 3D-array
    #      BEC(atom_id, BEC_x, BEC_y)
    ##=================================
    #
    from  ase.io.espresso import  read_espresso_out
    import numpy as np
    #
    ## unit conversion 
    # In ASE, unit is converted from QEoutput Ry/bohr to eV/Ang.
    # We have to back to QE units.
    # Ry: 13.605691932782346 eV
    # Bohr: 0.5291772083535413 Ang
    # above values are got from 
    # ==========================
    # from ase.units import create_units
    # units = create_units('2006')
    # ==========================
    conv=  0.5291772083535413/13.605691932782346 
    
    # BECのtxtデータの整形と保存
    test=np.loadtxt(directory+"/"+prefix)
    #print(test)
    
    # reshape to 3D-array
    NUM_ATOM=24
    BEC=test.reshape((NUM_ATOM,3,3))
    #print(BEC)

    #    
    # ASR:: calculate sum of BEC
    #####
    # (これがちょっと怪しいわ．．． 既に最小の桁になっちゃって誤差が丸まってるよねこれ．）
    #####
    BEC_SUM=np.sum(BEC, axis=0)
    #
    # ASR:: subtract BEC_SUM from BEC
    BEC=BEC-BEC_SUM/NUM_ATOM
    #
    # convert unit from Ry atomic units to Hretree atomic units.
    # e[Ry]=sqrt(2)[Hartree]
    #BEC=BEC/np.sqrt(2)
    #print(BEC)
    return BEC
    

In [46]:
# 1: 8ben_mergeからoutputファイルを読み込む．
# 2: 1構造1ファイルとして，8ben_BECに保存．
#
#
for count in range(0,11):
    # zero padding
    count_zero = str(count).zfill(2)
    print("STRUCTURE::  ", count_zero)
    output=extract_BEC_ph("2ben_out2/"+count_zero,"BEN2_"+count_zero+".txt")
    np.save("2ben_BEC/"+count_zero, output)
    

STRUCTURE::   00
STRUCTURE::   01
STRUCTURE::   02
STRUCTURE::   03
STRUCTURE::   04
STRUCTURE::   05
STRUCTURE::   06
STRUCTURE::   07
STRUCTURE::   08
STRUCTURE::   09
STRUCTURE::   10


In [18]:
t1=extract_BEC_ph("8ben_DFPT_out/0000","BEN2_0000.txt")
t2=extract_BEC_ph("8ben_DFPT_out/0100","BEN2_0100.txt")
print(t1-t2)

[[[-4.67854167e-03  1.71313542e-02  1.02555000e-01]
  [-3.79191667e-02 -1.45835417e-02 -6.76962500e-02]
  [ 3.98218750e-02  4.77876042e-02  2.87858333e-02]]

 [[ 9.98214583e-02  9.60513542e-02 -9.79950000e-02]
  [ 3.02083333e-03 -1.35035417e-02 -1.46762500e-02]
  [ 1.19118750e-02  6.25976042e-02 -1.21614167e-01]]

 [[-9.48854167e-03 -5.10586458e-02 -1.98950000e-02]
  [ 4.94508333e-02 -5.10354167e-03  5.65737500e-02]
  [-8.04581250e-02  8.02376042e-02  2.69958333e-02]]

 [[-9.48541667e-04  5.21913542e-02  8.18150000e-02]
  [ 3.01083333e-03  3.52464583e-02 -6.34062500e-02]
  [ 5.46818750e-02 -4.75923958e-02 -3.89041667e-02]]

 [[ 5.63145833e-03 -2.09386458e-02  4.63500000e-03]
  [ 1.96083333e-03 -4.85035417e-02 -3.88962500e-02]
  [-2.76881250e-02  1.33076042e-02  1.98258333e-02]]

 [[ 8.63145833e-03  2.44013542e-02 -3.82650000e-02]
  [ 4.30008333e-02  6.94645833e-03  7.98337500e-02]
  [ 1.88418750e-02 -9.47239583e-03 -1.29416667e-03]]

 [[-1.25185417e-02 -2.88286458e-02 -6.15150000e-02]
